In [0]:
%pip install -r requirements.txt
# %pip uninstall -y databricks-agents databricks-sdk
# %pip install --upgrade databricks-agents databricks-sdk
dbutils.library.restartPython()


  Obtaining dependency information for mlflow==2.21.2 from https://files.pythonhosted.org/packages/72/9c/08620b82d822eca5ed6054e8cc6d0fa4c46b2e883db9f7ddd3ac470339fb/mlflow-2.21.2-py3-none-any.whl.metadata
  Using cached mlflow-2.21.2-py3-none-any.whl.metadata (30 kB)
  Obtaining dependency information for databricks-agents==0.18.1 from https://files.pythonhosted.org/packages/0e/cc/0b0b7a07c6ed5f7af18e9d6dc39f28f41f1ffba06e6f7665e825857b000f/databricks_agents-0.18.1-py3-none-any.whl.metadata
  Using cached databricks_agents-0.18.1-py3-none-any.whl.metadata (3.5 kB)
  Obtaining dependency information for mlflow-skinny==2.21.2 from https://files.pythonhosted.org/packages/e2/ec/adfbccfac31e0d85003d74e03694a30ba54b5e0e3c7783de64952e9c3b84/mlflow_skinny-2.21.2-py3-none-any.whl.metadata
  Using cached mlflow_skinny-2.21.2-py3-none-any.whl.metadata (31 kB)
INFO: pip is looking at multiple versions of mlflow[databricks] to determine which version is compatible with other requirements. This cou

In [0]:
# interactive_chat.ipynb (Python script cells)

import mlflow
from chat_model import LangGraphChatAgent
from input_examples import input_example, input_examples
from config_utils import load_config
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksGenieSpace

# Load config values
genie_config = load_config("genie")
mlflow_config = load_config("mlflow")

GENIE_SPACE_ID = genie_config["genie_space_id"]
experiment_location = mlflow_config["experiment_location"]
uc_model = mlflow_config["uc_model"]
endpoint_name = mlflow_config["endpoint_name"]

Trace(request_id=tr-0f4fb0de8ef343ccb890f9c352b73ccb)

In [0]:
print(endpoint_name)

databricks-claude-3-7-sonnet


# Log model

In [0]:
mlflow.set_experiment(experiment_location)

# Define model resources
resources = [
    DatabricksServingEndpoint(endpoint_name=endpoint_name),
    DatabricksGenieSpace(genie_space_id=GENIE_SPACE_ID)
]

# Log model
with mlflow.start_run(run_name="db_ey_multiagent_genie_deploy_finaltest"):
    model_info = mlflow.pyfunc.log_model(
        python_model="chat_model.py",
        #LangGraphChatAgent(),
        artifact_path="genie_graph",
        input_example=input_example,
        code_paths=[
            "agent.py",
            "chat_model.py",
            "config_utils.py"
        ],
        pip_requirements="requirements.txt",
        resources=resources,
    )
    model_uri = model_info.model_uri
    print("✅ Model logged to:", model_uri)

2025/05/11 03:42:35 INFO mlflow.pyfunc: Predicting on input example to validate output


Uploading artifacts:   0%|          | 0/15 [00:00<?, ?it/s]

✅ Model logged to: runs:/9fb9ae8b96e14ba79bee62865401b733/genie_graph


# pre-deployment validation

In [0]:
import mlflow
from input_examples import input_example

# This assumes model_info is already defined from mlflow.pyfunc.log_model(...)
print("Running pre-deployment validation...")

# Run prediction test using the model_uri (e.g. runs:/...)
mlflow.models.predict(
    model_uri=model_info.model_uri,
    input_data=input_example
    #,
   # env_manager="uv"  # Optional: only if your environment requires it
)

print("Agent validation succeeded! Ready to register or deploy.")


2025/05/11 03:43:33 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.


Running pre-deployment validation...


2025/05/11 03:43:34 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2025/05/11 03:43:34 INFO mlflow.utils.virtualenv: Environment /tmp/virtualenv_envs/mlflow-7feea832e7b84653ac6482e1157d6bbedd1c8d14 already exists
2025/05/11 03:43:34 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-7feea832e7b84653ac6482e1157d6bbedd1c8d14/bin/activate && python -c ""']'
2025/05/11 03:43:34 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-7feea832e7b84653ac6482e1157d6bbedd1c8d14/bin/activate && python /local_disk0/.ephemeral_nfs/envs/pythonEnv-5ac1a891-a479-4c27-a277-95347ce1670a/lib/python3.11/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///local_disk0/user_tmp_data/spark-5ac1a891-a479-4c27-a277-95/tmpmd3k2m3c/genie_graph --content-type json --input-path /local_disk0/user_tmp_data/spark-5ac1a891-a479-4c27-a277-95/tmpkcqz4a43/input.json']'


{"messages": [{"role": "assistant", "content": "|    |   Total_Spend |\n|---:|--------------:|\n|  0 |        580538 |", "name": "Genie", "id": "551c17b8-377f-47fb-a195-b5361dce91b5"}, {"role": "assistant", "content": "# Goldfish Search Campaign Spending Analysis\n\n## Key Metrics\nThe total spend for Goldfish search campaigns in 2024 is $580,538.\n\n## Budget Insight\nThis significant investment of over half a million dollars represents a substantial commitment to Goldfish product marketing through search channels this year. This figure provides an important baseline for evaluating campaign performance and return on investment.\n\n## Recommendations\n- Compare this spend against conversion metrics to calculate cost per acquisition\n- Analyze spend distribution across different search platforms to identify efficiency opportunities\n- Consider evaluating spend by quarter to identify seasonal patterns\n- Benchmark against competitor spending if such data is available\n- Review historical

# register model to Unity Catalog

In [0]:
mlflow.set_registry_uri("databricks-uc")

catalog = "main"
schema = "retail_media"
model_name = "campbells_genai_versionFinal"

UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

uc_registered_model_info = mlflow.register_model(
    model_uri=model_uri,
    name=UC_MODEL_NAME
)


Registered model 'main.retail_media.campbells_genai_versionFinal' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/15 [00:00<?, ?it/s]

Created version '7' of model 'main.retail_media.campbells_genai_versionfinal'.


# Check the status of this model, should be 'Ready'

In [0]:
# stop here, don't forget to change the version in the following cells to the latest one!!!
aaa

In [0]:
from mlflow.tracking import MlflowClient

client = MlflowClient(registry_uri="databricks-uc")
version_info = client.get_model_version(
    name="main.retail_media.campbells_genai_versionFinal",

    version="7"
)
print("Status:", version_info.status)  # Should be "READY"


Status: READY


In [0]:
# due to version dependence difference, re-install a package here before deployment

In [0]:
%pip uninstall -y databricks-agents databricks-sdk
%pip install --upgrade databricks-agents databricks-sdk
dbutils.library.restartPython()

Found existing installation: databricks-agents 0.18.1
Uninstalling databricks-agents-0.18.1:
  Successfully uninstalled databricks-agents-0.18.1
Found existing installation: databricks-sdk 0.52.0
Uninstalling databricks-sdk-0.52.0:
  Successfully uninstalled databricks-sdk-0.52.0
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for databricks-agents from https://files.pythonhosted.org/packages/89/2f/86b47ea14fd09ad0e321e3c3173bacf4a115a57fbb5aeacfa34202269b08/databricks_agents-0.21.2-py3-none-any.whl.metadata
  Using cached databricks_agents-0.21.2-py3-none-any.whl.metadata (3.5 kB)
  Obtaining dependency information for databricks-sdk from https://files.pythonhosted.org/packages/80/ab/3bf8aaa39dad6c8b270176f60ae0f88928d3e16aeb2c6920dbd99ec280e2/databricks_sdk-0.52.0-py3-none-any.whl.metadata
  Using cached databricks_sdk-0.52.0-py3-none-any.whl.metadata (39 kB)
  Obtaining depe

In [0]:
# if the next cell fails, just try run the cell again. It will work during the second run.

# Deploy to endpoint and get the review app

In [0]:
from databricks import agents
# model details
UC_MODEL_NAME = "main.retail_media.???????????????"  # fill in here

UC_MODEL_VERSION = "7"

# Deploy to endpoint
deployment_info = agents.deploy(
    UC_MODEL_NAME,
    UC_MODEL_VERSION,
    #endpoint_name="agents_main-retail_media-campbells_genai_ey",  # optional but good to specify
    tags={"endpointSource": "campbells_deploy_ey"},
    environment_vars={
        "DATABRICKS_GENIE_PAT": "??????????????????"   # fill in here
           
    }
)

print("✅ Endpoint deployed at:", deployment_info.endpoint_url)



    Deployment of main.retail_media.campbells_genai_versionFinal version 7 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-3675705922506081.1.azuredatabricks.net/ml/endpoints/agents_main-retail_media-campbells_genai_versionFinal
    Review App: https://adb-3675705922506081.1.azuredatabricks.net/ml/review-v2/c1a2009f35584c18b4267e0574a549d8/chat
✅ Endpoint deployed at: https://adb-3675705922506081.1.azuredatabricks.net/ml/endpoints/agents_main-retail_media-campbells_genai_versionFinal


# Done!!!

# Example Use in notebook, 
## we are able to see the trace here in this notebook. But in the review app we don't show trace for clean interface.

In [0]:
# use this cell to trigger an error to stop. because we need to change the model_uri for the next cell, before running the next cell
aaa

---------------------------------------------------------------------------
BadRequest                                Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-5ac1a891-a479-4c27-a277-95347ce1670a/lib/python3.11/site-packages/databricks/agents/deployments.py:728, in deploy(model_name, model_version, scale_to_zero, environment_vars, instance_profile_arn, tags, workload_size, endpoint_name, budget_policy_id, **kwargs)
    723 try:
    724     # Here we don't need to update the ai-gateway config since using the put_ai_gateway throws an error
    725     # "Inference tables is already enabled". Also the ai-gateway is never part of the pending config,
    726     # it is associated with the endpoint directly.
    727     # Read https://github.com/databricks-eng/universe/pull/874814#discussion_r1920845945 for more info
--> 728     w.serving_endpoints.update_config(
    729         name=endpoint_name,
    730         served_entities=updated_served_model

In [0]:
import mlflow
from chat_model import LangGraphChatAgent
from mlflow.types.agent import ChatAgentMessage

AGENT = mlflow.pyfunc.load_model("??????????????????????") # copy paste the model_uri from the "Log Model" cell output 
response = AGENT.predict({
    "messages": [
        {
            "role": "user",
            "content": "If I want to invest an additional $1.4m on search campaigns, and my main goal is to attract impressions, which keywords should I target, please refer to search term level?"
                #"I have 100K budget for back to school this year. Which keywords should I invest in?"
        }
    ]
})
print("Final Answer:\n", response["messages"][-1]["content"])



2025/05/11 03:49:33 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.22.0, required: mlflow==2.21.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Final Answer:
 # Search Campaign Investment Strategy for Maximum Impressions

## Primary Keywords for Impression Maximization
Based on the search term data analysis, I recommend focusing your additional $1.4M budget on these high-impression keywords:

1. **"goldfish"** - Generates over 7.1M impressions with efficient spend ratio
2. **"goldfish crackers"** - Delivers 3M+ impressions, strong brand-specific search
3. **"goldfish crackers individual bags"** - 2.3M+ impressions with product specificity
4. **"snacks"** - Nearly 2M impressions with excellent efficiency

## Secondary Expansion Opportunities
These terms offer additional impression volume potential:
- **"goldfish snack packs"** (850K+ impressions)
- **"snacks variety pack for adults"** (840K+ impressions)
- **"crackers"** and **"gold fish"** (600K+ impressions each)

## Budget Allocation Recommendation
Distribute your $1.4M investment proportionally to impression potential, with approximately:
- 40% to "goldfish" and variations


[Trace(request_id=tr-321410651b7f4a3892ecbdfd02354db3), Trace(request_id=tr-cd9499b22018476c975d3da3b84626b9)]